In [1]:
import numpy as np
import torch
import gpytorch
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from torch.autograd import Variable

In [3]:
np.random.seed(0)

In [4]:
# Training data is 11 points in [0,1] inclusive regularly spaced
x = np.linspace(0,1,50)
x_tensor = Variable(torch.from_numpy(x).float())
all_index = np.arange(x.shape[0])

train_x_index = np.sort(np.random.choice(np.arange(x.shape[0]),11, replace=False))
H = np.zeros((x.shape[0], train_x_index.shape[0])).T

for q in range(train_x_index.shape[0]):
    for p in range(x.shape[0]):
        if train_x_index[q] == all_index[p]:
            H[q,p] = 1
H = H.T

train_x = x[train_x_index]
train_x = Variable(torch.from_numpy(train_x).float())

# True function is sin(2*pi*x) with Gaussian noise N(0,0.04)
train_y = Variable(torch.sin(train_x.data * (2 * np.pi)) + torch.randn(train_x.size()) * 0.2)

In [12]:
from torch import nn, optim
from gpytorch.kernels import RBFKernel, GridInterpolationKernel
from gpytorch.means import ConstantMean
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.random_variables import GaussianRandomVariable

In [13]:
# We use exact GP inference for regression
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean(constant_bounds=[-1e-5,1e-5])
        # Put a grid interpolation kernel over the RBF kernel
        self.base_covar_module = RBFKernel(log_lengthscale_bounds=(-5, 6))
        self.covar_module = GridInterpolationKernel(self.base_covar_module, grid_size=400,
                                                            grid_bounds=[(0, 1)])
        # Register kernel lengthscale as parameter
        self.register_parameter('log_outputscale', nn.Parameter(torch.Tensor([0])), bounds=(-5,6))
        
    def forward(self,x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        covar_x = covar_x.mul(self.log_outputscale.exp())
        return GaussianRandomVariable(mean_x, covar_x)

# initialize likelihood and model
likelihood = GaussianLikelihood(log_noise_bounds=(-5, 5))
# W = 1*np.ones((train_x.data.shape[0], train_x.data.shape[0]), dtype=np.float)
W = np.random.randn(x.shape[0], x.shape[0])
W = np.matmul(W.T,W)

model = GPRegressionModel(train_x.data, train_y.data, likelihood)

In [14]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.parameters()},  # Includes GaussianLikelihood parameters
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iter = 1000
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   log_lengthscale: %.3f   log_noise: %.3f' % (
        i + 1, training_iter, loss.data[0],
        model.covar_module.log_lengthscale.data[0, 0],
        model.likelihood.log_noise.data[0]
    ))
    optimizer.step()

AttributeError: 'GridInterpolationKernel' object has no attribute 'log_lengthscale'

In [ ]:
# Put model and likelihood into eval mode
model.eval()
likelihood.eval()

# Initialize plot
f, observed_ax = plt.subplots(1, 1, figsize=(4, 3))

# Calculate mean and predictive variance
observed_pred = likelihood(model(x_tensor))
# Labels are predictive means
pred_labels = observed_pred.mean().data.numpy()

# Define plotting function
def ax_plot(ax, rand_var, title):
    # Get upper and lower confidence bounds
    lower, upper = rand_var.confidence_region()
    # Plot training data as black stars
    ax.plot(train_x.data.numpy(), train_y.data.numpy(), 'k*')
    # Plot predictive means as blue line
    ax.plot(x_tensor.data.numpy(), rand_var.mean().data.numpy(), 'b')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(x_tensor.data.numpy(), lower.data.numpy(), upper.data.numpy(), alpha=0.5)
    ax.set_ylim([-3, 3])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])
    ax.set_title(title)
# Plot the predictions
ax_plot(observed_ax, observed_pred, 'Observed Values (Likelihood)')

In [ ]:
torch.cuda.is_available()


In [ ]:
torch.Tensor().cuda()

In [ ]:
observed_pred.mean().shape

In [ ]:
train_y.data.numpy().shape

In [ ]:
plt.plot(observed_pred.mean().data.numpy())